Note book to explore solutions.

Clean and remove null cells

In [170]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/movies_data_1.csv")

In [171]:

#get data from file data

# DATA CLEANING

#drop null columns
allColumns = ['MOVIES','YEAR','GENRE','RATING','ONE-LINE','STARS','VOTES','RunTime','Gross']


#df[allColumns] = df[allColumns].replace(['', ' ', 'nan', 'NaN'], pd.NA)

#df = df.dropna(subset=allColumns, how='all')
    #drop after
#df.dropna(subset=allColumns, inplace=True)
df = df.dropna(subset=allColumns, axis=0, how='all')
#df[allColumns] = df[allColumns].replace(r'^\s*$', np.nan, regex=True)



#select only some columns
nColumns = ['GENRE','ONE-LINE','STARS']
ownerColumn = ['owner_company']

#clean where you find \n and ''space
for col in nColumns:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace('\n', '', regex=False)
            .str.strip()
        )
# to clean only in owner column
for col in ownerColumn:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace('\t', '', regex=False)
            .str.strip()
        )
    #select all columns

print(df.head(25))

                                    MOVIES         YEAR  \
0                            Blood Red Sky       (2021)   
10     Masters of the Universe: Revelation     (2021– )   
11                        The Walking Dead  (2010–2022)   
12                          Rick and Morty     (2013– )   
13                         Army of Thieves       (2021)   
14                             Outer Banks     (2020– )   
34         The Last Letter from Your Lover       (2021)   
35                                  Dexter  (2006–2013)   
36                       Never Have I Ever     (2020– )   
37                            Virgin River     (2019– )   
38                     Gunpowder Milkshake       (2021)   
39                                 Lucifer  (2016–2021)   
40                       Fear Street: 1994       (2021)   
59                                Sex/Life     (2021– )   
60                   American Horror Story     (2011– )   
61                          Grey's Anatomy     (2005– ) 

In [172]:
#COLUMN SPLITING

print(df['STARS'].head(10))

#strip column names
df.columns = df.columns.str.strip()

#fill empty rows
df['STARS'] = df['STARS'].fillna('')

#split stars to director and starslist
df[['DIRECTOR','STARS-LIST']] = df['STARS'].str.split('|', n=1, expand=True)
df['STARS'] =df['STARS'].fillna('|')



#clean whitespace with str.strip()
df['DIRECTOR'] = df['DIRECTOR'].str.strip()
df['STARS-LIST'] = df['STARS-LIST'].str.strip()

#df['STARS-LIST'] = df['STARS-LIST'].replace('', np.nan)
print(df['DIRECTOR'].head(10))
print(df['STARS-LIST'].head(10))

df.drop(columns=['STARS'], inplace=True)


0     Director:Peter Thorwarth|     Stars:Peri Baume...
10    Stars:Chris Wood, Sarah Michelle Gellar, Lena ...
11    Stars:Andrew Lincoln, Norman Reedus, Melissa M...
12    Stars:Justin Roiland, Chris Parnell, Spencer G...
13    Director:Matthias Schweighöfer|     Stars:Matt...
14    Stars:Chase Stokes, Madelyn Cline, Madison Bai...
34    Director:Augustine Frizzell|     Stars:Shailen...
35    Stars:Michael C. Hall, Jennifer Carpenter, Dav...
36    Stars:Maitreyi Ramakrishnan, Poorna Jagannatha...
37    Stars:Alexandra Breckenridge, Martin Henderson...
Name: STARS, dtype: object
0                              Director:Peter Thorwarth
10    Stars:Chris Wood, Sarah Michelle Gellar, Lena ...
11    Stars:Andrew Lincoln, Norman Reedus, Melissa M...
12    Stars:Justin Roiland, Chris Parnell, Spencer G...
13                       Director:Matthias Schweighöfer
14    Stars:Chase Stokes, Madelyn Cline, Madison Bai...
34                          Director:Augustine Frizzell
35    Stars:Michael C

In [ ]:
print(df.head(10))

In [173]:
df['Extract_date'] = pd.to_datetime(df['Extract_date'],errors='coerce') #to_datetime converts it to a object

#create column from extact date and time
df['extraction_date'] = df['Extract_date'].dt.date
df['extraction_time'] = df['Extract_date'].dt.time

print(df[['extraction_time','extraction_date']].head())

df.drop(columns=['Extract_date'], inplace=True)


    extraction_time extraction_date
0          00:00:00      2020-01-01
10  04:16:53.364872      2020-01-01
11  04:42:34.701360      2020-01-01
12  05:08:16.037847      2020-01-01
13  05:33:57.374334      2020-01-01


In [174]:
#YEAR CALCULATIONS

#convert to string
#df['YEAR'] = df['YEAR'].astype(str)

#to fix index a
df.reset_index(drop=True, inplace=True)

df['YEAR'] = df['YEAR'].fillna('')

df['YEAR'] = df['YEAR'].str.replace(r'\(I\)', '', regex=True)
df['YEAR'] = df['YEAR'].str.replace(r'\(II\)', '', regex=True)
df['YEAR'] = df['YEAR'].str.replace(r'\(III\)', '', regex=True)

df['YEAR'] = df['YEAR'].str.replace(r'\(', '', regex=True)
df['YEAR'] = df['YEAR'].str.replace(r'\)', '', regex=True)

df['YEAR'] = df['YEAR'].str.replace('–', '-')

df['YEAR'] = df['YEAR'].str.replace(r'[^\d-]', '', regex=True)

df['YEAR'] = df['YEAR'].str.strip()

df['YEAR'] = df['YEAR'].replace('', '-')

df[['start_year','end_year']] = df['YEAR'].str.split('-',n=1, expand=True)

df['end_year'] = df['end_year'].fillna(df['start_year'])

df['end_year'] = df['end_year'].replace('', 'present')

#run a funciton with .apply to each row(axis=1)
#df['duration_years'] = df.apply(lambda row: int(row['end_year']) - int(row['start_year']) + 1 if row['end_year'] != 'present' else pd.NA, axis=1)

durations = []

for i in range(len(df)):
    start = df.loc[i,'start_year']
    end = df.loc[i,'end_year']

    if end == 'present':
        durations.append(pd.NA)
    else:
        durations.append(int(end) - int(start) + 1)

df['duration'] = durations

df.drop(columns=['YEAR'], inplace=True)




In [180]:
#dimcompany is probably owner_company
#dimdirector is probably director

unique_companies = df['owner_company'].dropna().unique()

DimCompan = pd.DataFrame(unique_companies, columns=['DimCompan'])
print(DimCompan.head())


unique_directors = df['DIRECTOR'].dropna().unique()

DimDirector = pd.DataFrame(unique_directors, columns=['DimDirector'])
print(DimDirector.head())

DimCompan.to_csv(f"../output/uniqueCompany.csv",index=False)
DimDirector.to_csv(f"../output/uniqueDirector.csv",index=False)


                 DimCompan
0        Columbia Pictures
1  Legendary Entertainment
2       Universal Pictures
3       Paramount Pictures
4     Walt Disney Pictures
                                         DimDirector
0                           Director:Peter Thorwarth
1  Stars:Chris Wood, Sarah Michelle Gellar, Lena ...
2  Stars:Andrew Lincoln, Norman Reedus, Melissa M...
3  Stars:Justin Roiland, Chris Parnell, Spencer G...
4                     Director:Matthias Schweighöfer


In [176]:
print(df.head(10))

                                MOVIES                         GENRE  RATING  \
0                        Blood Red Sky      Action, Horror, Thriller     6.1   
1  Masters of the Universe: Revelation  Animation, Action, Adventure     5.0   
2                     The Walking Dead       Drama, Horror, Thriller     8.2   
3                       Rick and Morty  Animation, Adventure, Comedy     9.2   
4                      Army of Thieves         Action, Crime, Horror     NaN   
5                          Outer Banks          Action, Crime, Drama     7.6   
6      The Last Letter from Your Lover                Drama, Romance     6.8   
7                               Dexter         Crime, Drama, Mystery     8.6   
8                    Never Have I Ever                        Comedy     7.9   
9                         Virgin River                Drama, Romance     7.4   

                                            ONE-LINE    VOTES  RunTime Gross  \
0  A woman with a mysterious illness is

In [182]:
#to check if data is changed correctly

from datetime import datetime

timeStamp = datetime.now().strftime("%H%M%S")

#df.to_csv(f"../output/secondOutputAtTime{timeStamp}.csv",index=False)
#df.to_csv(f"../output/outputDataCleaned.csv",index=False)
#df.to_csv(f"../output/outputAtTime{timeStamp}.csv",index=False)
df.to_excel("../output/finalOutput.xlsx",index=False)